<a href="https://colab.research.google.com/github/srutibanta/Simplifying-Sentimate-Analysis-using-python/blob/main/Simpliflying_Sentiment_Analysis_using_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install vaderSentiment

     |████████████████████████████████| 133kB 12.6MB/s 


In [2]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
vs=SentimentIntensityAnalyzer()


In [3]:
text='The product is really good'
vs.polarity_scores(text)

{'compound': 0.4927, 'neg': 0.0, 'neu': 0.556, 'pos': 0.444}

# **Web Scrapping**

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os

In [5]:
url='https://inshorts.com/en/read/sports'
news_data=[]
news_category=url.split('/')[-1]
data=requests.get(url)
soup=BeautifulSoup(data.content)
print(soup)

<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<style>
    /* The Modal (background) */
    .modal_contact {
        display: none; /* Hidden by default */
        position: fixed; /* Stay in place */
        z-index: 8; /* Sit on top */
        left: 0;
        top: 0;
        width: 100%; /* Full width */
        height: 100%;
        overflow: auto; /* Enable scroll if needed */
        background-color: rgb(0,0,0); /* Fallback color */
        background-color: rgba(0,0,0,0.4); /* Black w/ opacity */
    }

    /* Modal Content/Box */
    .modal-content {
        background-color: #fefefe;
        margin: 15% auto;
        padding: 20px !important;
        padding-top: 0 !important;
        /* border: 1px solid #888; */
        text-align: center;
        position: relative;
        border-radius: 6px;
    }

    /* The Close Button */
    .close {
      left: 90%;
      color: #aaa;
      float: right;
      font-size: 28px;
      font-weight: bold;
    /* positio

In [6]:
urls=['https://inshorts.com/en/read/sports',
      'https://inshorts.com/en/read/world',
      'https://inshorts.com/en/read/politics']
def build_dataset(urls):
  news_data=[]
  for url in urls:
    news_category=url.split('/')[-1]
    data=requests.get(url)
    soup=BeautifulSoup(data.content)
    news_articles=[{'news_headline':headline.find('span',attrs={"itemprop":"headline"}).string,
                    'news_article':article.find('div',attrs={"itemprop":"articleBody"}).string,
                    'news_category':news_category}
                   for headline,article in zip(soup.find_all('div',class_=["news-card-title news-right-box"]),
                                               soup.find_all('div',class_=["news-card-content news-right-box"]))]

  
                                                               
    news_articles=news_articles[0:20]
    news_data.extend(news_articles)
  df=pd.DataFrame(news_data)
  df=df[['news_headline','news_article','news_category']]
  return df
 

    

In [7]:
df=build_dataset(urls)
df.head()


,news_headline,news_article,news_category
0,"People don't know how BCCI works, they talk r...",After BCCI’s communication with media regardin...,sports
1,PSL players stand in shape of 'D' to pay tribu...,Ahead of the Pakistan Super League Qualifier b...,sports
2,"Harsha Bhogle names his Team of IPL 2020, incl...",Commentator Harsha Bhogle picked his Team of I...,sports
3,Sachin Tendulkar and Brian Lara play golf toge...,Ex-cricketers Sachin Tendulkar and Brian Lara ...,sports
4,Misbah saab hi bata sakte hain: Amir after jou...,A journalist took to Twitter to question fast ...,sports


In [8]:
df.to_csv('news.csv',index=False)

In [9]:
import nltk
nltk.download('stopwords')
stopword_list=nltk.corpus.stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')
len(stopword_list)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


177

# **Remove Html Tag**

In [10]:
def html_tag(text):
  soup=BeautifulSoup(text,"html.parser")
  new_text=soup.get_text()
  return new_text
html_tag('<html><h2> Some important info </h2><html>')

' Some important info '

# **Expand Contractions**

In [11]:
!pip install contractions
import contractions
def con(text):
  expand=contractions.fix(text)
  return expand
con("Y'all can't expand I'd Think")

     |████████████████████████████████| 245kB 12.3MB/s 
     |████████████████████████████████| 317kB 11.6MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81701 sha256=cb7b3eb8b113724061823513de9cfcfc304424adcb9eb1b8c5d376ef8cf3f412
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


'you all can not expand I would Think'

# **special characters**

In [12]:
import re 
def remove_sp(text):
  pattern=r'[^A-Za-z0-9\s]'
  text=re.sub(pattern,'',text)
  return text
remove_sp("Well this was fun! What do  you think? 123#")

'Well this was fun What do  you think 123'

In [13]:
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer=ToktokTokenizer()

In [14]:
def remove_stopwords(text):
  tokens=tokenizer.tokenize(text)
  tokens=[token.strip() for token in tokens]
  filtered_tokens=[token for token in tokens if token  not in stopword_list]
  filtered_text=' '.join(filtered_tokens)
  return filtered_text

In [15]:
remove_stopwords("The, and , if are all stopwords and even not")

'The , , stopwords even not'

In [16]:
df.news_headline=df.news_headline.apply(lambda x:x.lower())
df.news_article=df.news_article.apply(lambda x:x.lower())
df.news_headline=df.news_headline.apply(html_tag)
df.news_article=df.news_article.apply(html_tag)
df.news_headline=df.news_headline.apply(con)
df.news_article=df.news_article.apply(con)
df.news_headline=df.news_headline.apply(remove_sp)
df.news_article=df.news_article.apply(remove_sp)
df.news_headline=df.news_headline.apply(remove_stopwords)
df.news_article=df.news_article.apply(remove_stopwords)
df.head()



,news_headline,news_article,news_category
0,people not know bcci works talk rubbish gangul...,bccis communication media regarding injuries c...,sports
1,psl players stand shape would pay tribute dean...,ahead pakistan super league qualifier multan s...,sports
2,harsha bhogle names team ipl 2020 includes 4 p...,commentator harsha bhogle picked team ipl 2020...,sports
3,sachin tendulkar brian lara play golf together...,excricketers sachin tendulkar brian lara playe...,sports
4,misbah saab hi bata sakte hain amir journo que...,journalist took twitter question fast bowler m...,sports


In [17]:
df['compound']=df['news_article'].apply(lambda x:vs.polarity_scores(x)['compound'])

In [18]:
df.head()

,news_headline,news_article,news_category,compound
0,people not know bcci works talk rubbish gangul...,bccis communication media regarding injuries c...,sports,-0.6808
1,psl players stand shape would pay tribute dean...,ahead pakistan super league qualifier multan s...,sports,0.0516
2,harsha bhogle names team ipl 2020 includes 4 p...,commentator harsha bhogle picked team ipl 2020...,sports,0.5267
3,sachin tendulkar brian lara play golf together...,excricketers sachin tendulkar brian lara playe...,sports,0.9709
4,misbah saab hi bata sakte hain amir journo que...,journalist took twitter question fast bowler m...,sports,0.3400
